In [63]:
import pandas as pd
import numpy as np

In [77]:
data_dir = 'data/mimic-iv-1.0/physionet.org/files/mimiciv/1.0/'
core = data_dir+'core/' 
hosp = data_dir+'hosp/'
icu = data_dir+'icu/'

In [78]:
patients = pd.read_csv(core+'patients.csv')
admissions = pd.read_csv(core+'admissions.csv')
transfers = pd.read_csv(core+'transfers.csv')
drgs = pd.read_csv(hosp+'drgcodes.csv')

In [79]:
neonates = admissions.merge(patients, on="subject_id")
neonates = neonates[neonates["anchor_age"] == 0]
print(f"Total neonate admissions: {len(neonates)}")

Total neonates: 61628


In [81]:
nicu_transfers = transfers[transfers['careunit'] == 'Neonatal Intensive Care Unit (NICU)']
print(f"Total NICU transfers: {len(nicu_transfers)}")

Total NICU transfers: 29363


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
73784,14312010,29997871.0,32892824,admit,Neonatal Intensive Care Unit (NICU),2118-10-25 02:07:31,2118-10-25 04:14:02
73786,14312010,29997871.0,30037787,transfer,Neonatal Intensive Care Unit (NICU),2118-10-25 04:14:02,2118-10-25 04:50:33
73789,15023067,27017665.0,30129282,transfer,Neonatal Intensive Care Unit (NICU),2127-11-27 05:28:31,2127-11-27 06:21:38
73790,15023067,27017665.0,35738088,transfer,Neonatal Intensive Care Unit (NICU),2127-11-27 06:21:38,2127-11-27 07:41:44
73804,19197569,29780270.0,39194191,transfer,Neonatal Intensive Care Unit (NICU),2185-04-01 01:22:57,2185-04-01 02:32:02
...,...,...,...,...,...,...,...
2132036,18064853,20512742.0,34898112,admit,Neonatal Intensive Care Unit (NICU),2183-11-15 18:35:40,2183-11-15 19:41:53
2132256,12951316,21588163.0,34133788,transfer,Neonatal Intensive Care Unit (NICU),2161-07-20 03:33:07,2161-07-20 04:20:23
2132299,12630902,23888754.0,36223364,transfer,Neonatal Intensive Care Unit (NICU),2162-11-09 03:54:55,2162-11-09 04:07:06
2132357,13366767,26811822.0,36654815,transfer,Neonatal Intensive Care Unit (NICU),2133-11-01 00:29:11,2133-11-01 03:24:32


In [88]:
neonates_nicu = pd.merge(neonates, nicu_transfers, on=['subject_id', 'hadm_id'], how='inner')
neonates_nicu = neonates_nicu[neonates_nicu['eventtype'] == 'admit']
print(f"Neonates with NICU transfers: {len(neonates_nicu)}")

Neonates with NICU transfers: 7758


In [89]:
neonates_nicu['hospital_expire_flag'].value_counts()

hospital_expire_flag
0    7617
1     141
Name: count, dtype: int64

In [48]:
premature_drg_codes = [791, 792]
premature_drg = drgs[drgs['drg_code'].isin(premature_drg_codes)]
premature = pd.merge(patients, premature_drg, on=['subject_id'], how='inner')
print(f"Total premature: {len(premature)}")

Total premature: 4951


In [52]:
premature_mort = pd.merge(premature, admissions, on=['hadm_id'], how='inner')
premature_mort['discharge_location'].value_counts()

discharge_location
HOME                        3038
HOME HEALTH CARE            1861
SKILLED NURSING FACILITY      25
ACUTE HOSPITAL                 9
HEALTHCARE FACILITY            7
REHAB                          4
AGAINST ADVICE                 1
PSYCH FACILITY                 1
HOSPICE                        1
Name: count, dtype: int64

In [61]:
neonate_mort = pd.merge(neonates, admissions, on=['subject_id'], how='inner')
neonate_mort = neonate_mort[neonate_mort['hospital_expire_flag'] == 1]

In [62]:
neonate_mort_drg = pd.merge(neonate_mort, drgs, on=['hadm_id'], how='left')
neonate_mort_drg['description'].value_counts()

description
NEONATES, DIED OR TRANSFERRED TO ANOTHER ACUTE CARE FACILITY                                                                          207
NEONATE BWT < 500G, OR BWT 500-999G & GESTATIONAL AGE <24 WKS, OR BWT 500-749G W MAJOR ANOMALY OR W/O LIFE SUSTAINING INTERVENTION     76
NEONATE BIRTHWT 500-749G W/O MAJOR PROCEDURE                                                                                           47
NEONATE BIRTHWT 750-999G W/O MAJOR PROCEDURE                                                                                           17
NEONATE BWT 2000-2499G W MAJOR ANOMALY                                                                                                 13
NEONATE BWT 1000-1249G W RESP DIST SYND/OTH MAJ RESP OR MAJ ANOM                                                                       11
NEONATE BIRTHWT >2499G W MAJOR ANOMALY                                                                                                  8
NEONATE BIRTHWT 1500-1